In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import json

In [44]:
api_key = "T4UQ50NF71Q6Y0K4NVMZ"

외환보유액


In [192]:
import requests
import pandas as pd

def get_foreign_reserve():
    api_key = "T4UQ50NF71Q6Y0K4NVMZ"
    stat_code = "732Y001"
    url = f"https://ecos.bok.or.kr/api/StatisticSearch/{api_key}/json/kr/1/100000/{stat_code}/M/197101/202412"

    res = requests.get(url)
    data = res.json()

    if "StatisticSearch" in data and "row" in data["StatisticSearch"]:
        df = pd.DataFrame(data["StatisticSearch"]["row"])
        df = df[df["ITEM_NAME1"] == "합계"][["TIME", "DATA_VALUE"]]
        df.rename(columns={"TIME": "날짜", "DATA_VALUE": "외환보유액(천달러)"}, inplace=True)
        df["날짜"] = pd.to_datetime(df["날짜"], format="%Y%m")
        return df
    else:
        print("데이터가 없습니다:", data)
        return pd.DataFrame()

# 실행
df_foreign_reserve = get_foreign_reserve()
df_foreign_reserve

,날짜,외환보유액(천달러)
4,1971-01-01,593845
9,1971-02-01,566473
14,1971-03-01,570047
19,1971-04-01,570295
24,1971-05-01,573831
...,...,...
3219,2024-08-01,415916439
3224,2024-09-01,419969772
3229,2024-10-01,415689646
3234,2024-11-01,415391699


In [193]:
#외환보유액 일로 뻥튀기
import pandas as pd

def expand_monthly_to_daily(df_monthly):
    df_monthly = df_monthly.copy().reset_index(drop=True)  # ✅ 인덱스 초기화
    df_monthly["START_DATE"] = df_monthly["날짜"]

    expanded = []

    # 마지막 전까지 반복
    for i in range(len(df_monthly) - 1):
        start = df_monthly.loc[i, "START_DATE"]
        end = df_monthly.loc[i + 1, "START_DATE"] - pd.Timedelta(days=1)
        dates = pd.date_range(start, end)
        for d in dates:
            expanded.append({
                "DATE": d,
                "외환보유액(천달러)": df_monthly.loc[i, "외환보유액(천달러)"]
            })

    # 마지막 월 처리
    last_start = df_monthly.loc[len(df_monthly) - 1, "START_DATE"]
    last_end = last_start + pd.offsets.MonthEnd(0)
    last_dates = pd.date_range(last_start, last_end)
    for d in last_dates:
        expanded.append({
            "DATE": d,
            "외환보유액(천달러)": df_monthly.loc[len(df_monthly) - 1, "외환보유액(천달러)"]
        })

    df_daily = pd.DataFrame(expanded)
    return df_daily

# 사용 예시
df_foreign_reserve_daily = expand_monthly_to_daily(df_foreign_reserve)

In [194]:
df_foreign_reserve_daily

,DATE,외환보유액(천달러)
0,1971-01-01,593845
1,1971-01-02,593845
2,1971-01-03,593845
3,1971-01-04,593845
4,1971-01-05,593845
...,...,...
19719,2024-12-27,415603777
19720,2024-12-28,415603777
19721,2024-12-29,415603777
19722,2024-12-30,415603777


In [83]:
import requests
import pandas as pd

api_key = "T4UQ50NF71Q6Y0K4NVMZ"
stat_code = "732Y001"
url = f"https://ecos.bok.or.kr/api/StatisticItemList/{api_key}/json/kr/1/1000/{stat_code}"

res = requests.get(url)
data = res.json()

if "StatisticItemList" in data and "row" in data["StatisticItemList"]:
    df_items = pd.DataFrame(data["StatisticItemList"]["row"])
    print(df_items[["ITEM_CODE", "ITEM_NAME"]].drop_duplicates())
else:
    print("항목 목록 조회 실패:", data)

   ITEM_CODE ITEM_NAME
0         01         금
3         02     특별인출권
6         03    IMF포지션
9         04        외환
12        99        합계


In [182]:
import requests
import pandas as pd

stat_code = "732Y001"
item_codes = {
    "01": "금",
    "02": "특별인출권",
    "03": "IMF포지션",
    "04": "외환",
    "99": "외환보유액(합계)"
}
start_date = "199401"
end_date = "202412"

dfs = []

for code, name in item_codes.items():
    url = f"https://ecos.bok.or.kr/api/StatisticSearch/{api_key}/json/kr/1/100000/{stat_code}/M/{start_date}/{end_date}/{code}"
    res = requests.get(url)
    data = res.json()

    if "StatisticSearch" in data and "row" in data["StatisticSearch"]:
        df = pd.DataFrame(data["StatisticSearch"]["row"])
        df = df[["TIME", "DATA_VALUE"]]
        df.rename(columns={"DATA_VALUE": name}, inplace=True)
        dfs.append(df)


df_merged_in = dfs[0]
for df in dfs[1:]:
    df_merged_in = pd.merge(df_merged_in, df, on="TIME", how="outer")


print(df_merged_in)

       TIME        금     특별인출권   IMF포지션         외환  외환보유액(합계)
0    199401    33280     58455   477055   19975629   20544420
1    199402    33280     62218   468871   20179177   20743546
2    199403    33280     62842   480806   20178054   20754982
3    199404    33280     63363   517202   20399986   21013831
4    199405    33524     66521   498809   20803922   21402776
..      ...      ...       ...      ...        ...        ...
367  202408  4794759  15247963  4388198  391485518  415916439
368  202409  4794759  15329419  4266655  395578939  419969772
369  202410  4794759  15047408  4175346  391672133  415689646
370  202411  4794759  14897111  4185169  391514661  415391699
371  202412  4794759  14714138  4204947  391889933  415603777

[372 rows x 6 columns]


대외채권

In [95]:
import requests
import pandas as pd

api_key = "T4UQ50NF71Q6Y0K4NVMZ"
stat_code = "311Y005"
start = 1
end = 100000
period = "Q"
start_date = "1994Q1"
end_date = "2024Q4"

url = f"https://ecos.bok.or.kr/api/StatisticSearch/{api_key}/json/kr/{start}/{end}/{stat_code}/{period}/{start_date}/{end_date}"
response = requests.get(url)
data = response.json()

if "StatisticSearch" in data and "row" in data["StatisticSearch"]:
    df = pd.DataFrame(data["StatisticSearch"]["row"])
    df_bond = df[["ITEM_NAME1", "TIME", "DATA_VALUE"]]
    df_bond = df_bond.pivot(index="TIME", columns="ITEM_NAME1", values="DATA_VALUE")
    df_bond = df_bond.reset_index()
    print(df_bond.head())
else:
    print("❌ 데이터가 없습니다 또는 응답 오류:", data)

ITEM_NAME1    TIME 1.1.증권투자 1.2.대출금 1.3.현금및예금 1.4.무역신용 1.5.기타자산 1.6.준비자산  \
0           1994Q4    425.4  4129.2    5525.1   3182.7  17601.6    22340   
1           1995Q1    486.3  5665.6    6030.5   3029.6  18718.1    23210   
2           1995Q2    529.1  4994.2    6627.7   3476.8  20157.2    25020   
3           1995Q3    582.7  5668.7    7269.4   3401.8  21358.9    27830   
4           1995Q4    582.2  6092.6    7683.9   3554.5  22597.3    29350   

ITEM_NAME1     1.단기 2.1.증권투자 2.2.대출금  ... D-나.2.1.증권투자 D-나.2.2.대출금  \
0             53204   3962.2  5476.2  ...            0           0   
1           57140.2   4375.3  5894.2  ...            0           0   
2             60805   5305.7  6492.2  ...            0           0   
3           66111.5   5926.9  7093.6  ...          1.8           0   
4           69860.5   6175.2  7734.8  ...          0.3           0   

ITEM_NAME1 D-나.2.3.현금및예금 D-나.2.4.무역신용 D-나.2.5.기타자산 D-나.2.6.채무상품 직접투자 D-나.2.장기  \
0                      0        241.7    



---



대외채무

In [104]:
import requests
import pandas as pd

api_key = "T4UQ50NF71Q6Y0K4NVMZ"
stat_code = "311Y004"
start = 1
end = 100000
period = "Q"
start_date = "1994Q1"
end_date = "2024Q4"

url = f"https://ecos.bok.or.kr/api/StatisticSearch/{api_key}/json/kr/{start}/{end}/{stat_code}/{period}/{start_date}/{end_date}"
response = requests.get(url)
data = response.json()

if "StatisticSearch" in data and "row" in data["StatisticSearch"]:
    df = pd.DataFrame(data["StatisticSearch"]["row"])
    df_bond = df[["ITEM_NAME1", "TIME", "DATA_VALUE"]]
    df_bond = df_bond.pivot(index="TIME", columns="ITEM_NAME1", values="DATA_VALUE")
    df_bond = df_bond.reset_index()
    print(df_bond.head())
else:
    print("❌ 데이터가 없습니다 또는 응답 오류:", data)

ITEM_NAME1    TIME 1.1.증권발행  1.2.차입금 1.3.현금및예금 1.4.무역신용 1.5.기타부채     1.단기  \
0           1994Q4        0  34428.8     656.8    821.5    277.5  36184.6   
1           1995Q1        0    39625    1107.8      888      326  41946.8   
2           1995Q2        0  43157.7      1457    958.7    340.8  45914.1   
3           1995Q3        0    47549    1520.8    948.4    336.8    50355   
4           1995Q4    209.4  48298.5    1543.9    955.6    355.6  51363.1   

ITEM_NAME1 2.1.증권발행        2.2.차입금          2.3.현금및예금  ... D-나.2.1.증권발행  \
0                  19556.9          24520.9         0  ...       8844.7   
1                  22185.7          24632.4         0  ...       9187.8   
2                  25641.3          25683.4         0  ...      10469.1   
3                    27520          26438.2         0  ...        10744   
4                  29559.6          27470.7         0  ...      11209.1   

ITEM_NAME1 D-나.2.2.차입금 D-나.2.3.현금및예금 D-나.2.4.무역신용 D-나.2.5.기타부채  \
0               9486

In [43]:
import requests
import pandas as pd

start = 1
end = 1000
statcode = '311Y004'
cycle = 'A'
time1 = '1994'
time2 = '2024'
url = f"http://ecos.bok.or.kr/api/StatisticSearch/{api_key}/json/kr/{start}/{end}/{statcode}/{cycle}/{time1}/{time2}"
response = requests.get(url)
data = response.json()

df = pd.DataFrame(data['StatisticSearch']['row'])
df_debt = df[['TIME', 'DATA_VALUE']]
df_debt.rename(columns={'DATA_VALUE': '대외채무'}, inplace=True)
df_debt

<ipython-input-43-8724fb7f904e>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_debt.rename(columns={'DATA_VALUE': '대외채무'}, inplace=True)


,TIME,대외채무
0,1994,80894.7
1,1994,7187.7
2,1994,0
3,1994,0
4,1994,0
...,...,...
995,2004,1368.7
996,2004,425.5
997,2004,310.9
998,2004,0




---



순대외채권

In [98]:
import requests
import pandas as pd

api_key = "T4UQ50NF71Q6Y0K4NVMZ"
stat_code = "311Y006"
start = 1
end = 100000
period = "Q"
start_date = "1994Q1"
end_date = "2024Q4"

url = f"https://ecos.bok.or.kr/api/StatisticSearch/{api_key}/json/kr/{start}/{end}/{stat_code}/{period}/{start_date}/{end_date}"
response = requests.get(url)
data = response.json()

if "StatisticSearch" in data and "row" in data["StatisticSearch"]:
    df = pd.DataFrame(data["StatisticSearch"]["row"])
    df_bond = df[["ITEM_NAME1", "TIME", "DATA_VALUE"]]
    df_bond = df_bond.pivot(index="TIME", columns="ITEM_NAME1", values="DATA_VALUE")
    df_bond = df_bond.reset_index()
    print(df_bond.head())
else:
    print("❌ 데이터가 없습니다 또는 응답 오류:", data)

ITEM_NAME1    TIME     1.단기      2.장기 A-1.단기   A-2.장기   A.일반정부   B-1.단기  \
0           1994Q4  17019.4  -33787.3  531.5  -7047.9  -6516.3    25553   
1           1995Q1  15193.3  -35569.2  509.7  -6845.1  -6335.4  26446.5   
2           1995Q2  14890.9  -38332.9    351  -7263.3  -6912.3  28304.8   
3           1995Q3  15756.5  -39540.3  542.2  -7191.3  -6649.1    31114   
4           1995Q4  18497.4  -41620.8  370.7  -6449.5  -6078.8  32650.5   

ITEM_NAME1  B-2.장기   B.중앙은행    C-1.단기  ...   D-1.단기    D-2.장기 D-가.1.단기  \
0             -152    25401   -8017.2  ...  -1047.9  -17695.9    153.5   
1           -206.4    26240  -10231.3  ...  -1531.6    -18205    241.4   
2           -209.7  28095.1  -12709.6  ...  -1055.3  -19844.3    208.4   
3           -204.8  30909.2  -14882.2  ...  -1017.6    -20737    197.7   
4           -145.2  32505.3  -13882.8  ...   -640.9  -21847.4    214.3   

ITEM_NAME1 D-가.2.장기 D-가.기타금융기관 D-나.1.단기  D-나.2.장기 D-나.비금융기업 등    D.기타부문  \
0              82.7      236.

In [93]:
import requests
import pandas as pd

api_key = "T4UQ50NF71Q6Y0K4NVMZ"

start = 1
end = 1000
statcode = '311Y006'
cycle = 'A'
time1 = '1994'
time2 = '2024'
url = f"http://ecos.bok.or.kr/api/StatisticSearch/{api_key}/json/kr/{start}/{end}/{statcode}/{cycle}/{time1}/{time2}"
response = requests.get(url)
data = response.json()

df = pd.DataFrame(data['StatisticSearch']['row'])
df_net = df[['TIME', 'DATA_VALUE']]
df_net.rename(columns={'DATA_VALUE': '순대외채권'}, inplace=True)
df_net

<ipython-input-93-f6d569a6890a>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_net.rename(columns={'DATA_VALUE': '순대외채권'}, inplace=True)


,TIME,순대외채권
0,1994,-16767.9
1,1994,-6516.3
2,1994,531.5
3,1994,-7047.9
4,1994,25401
...,...,...
646,2024,-12043
647,2024,19162.8
648,2024,-31205.8
649,2024,480149.3




---


대외채권 일별

In [106]:
import pandas as pd
import requests

api_key = "T4UQ50NF71Q6Y0K4NVMZ"
stat_code = "311Y005"
start = 1
end = 100000
period = "Q"
start_date = "1994Q1"
end_date = "2024Q4"

url = f"https://ecos.bok.or.kr/api/StatisticSearch/{api_key}/json/kr/{start}/{end}/{stat_code}/{period}/{start_date}/{end_date}"
response = requests.get(url)
data = response.json()

if "StatisticSearch" in data and "row" in data["StatisticSearch"]:
    df = pd.DataFrame(data["StatisticSearch"]["row"])
    df_bond = df[["ITEM_NAME1", "TIME", "DATA_VALUE"]]
    df_bond = df_bond.pivot(index="TIME", columns="ITEM_NAME1", values="DATA_VALUE").reset_index()
    df_bond["START_DATE"] = pd.to_datetime(df_bond["TIME"].str[:4] + "-" +
                                           df_bond["TIME"].str[4:].replace({'Q1': '01', 'Q2': '04', 'Q3': '07', 'Q4': '10'}) +
                                           "-01")

    # 분기 데이터를 일별로 확장
    expanded_rows = []
    for i in range(len(df_bond) - 1):
        start = df_bond.loc[i, 'START_DATE']
        end = df_bond.loc[i + 1, 'START_DATE'] - pd.Timedelta(days=1)
        dates = pd.date_range(start, end)
        for d in dates:
            row = df_bond.iloc[i].drop(["TIME", "START_DATE"])
            row_data = {"DATE": d}
            row_data.update(row.to_dict())
            expanded_rows.append(row_data)

    # 마지막 분기 처리
    last_start = df_bond.iloc[-1]['START_DATE']
    last_dates = pd.date_range(last_start, last_start + pd.offsets.QuarterEnd(0))
    for d in last_dates:
        row = df_bond.iloc[-1].drop(["TIME", "START_DATE"])
        row_data = {"DATE": d}
        row_data.update(row.to_dict())
        expanded_rows.append(row_data)

    # 최종 일별 데이터프레임 생성
    df_daily_bond = pd.DataFrame(expanded_rows)
    print(df_daily_bond.head())
else:
    print("❌ 데이터가 없습니다 또는 응답 오류:", data)

        DATE 1.1.증권투자 1.2.대출금 1.3.현금및예금 1.4.무역신용 1.5.기타자산 1.6.준비자산   1.단기  \
0 1994-10-01    425.4  4129.2    5525.1   3182.7  17601.6    22340  53204   
1 1994-10-02    425.4  4129.2    5525.1   3182.7  17601.6    22340  53204   
2 1994-10-03    425.4  4129.2    5525.1   3182.7  17601.6    22340  53204   
3 1994-10-04    425.4  4129.2    5525.1   3182.7  17601.6    22340  53204   
4 1994-10-05    425.4  4129.2    5525.1   3182.7  17601.6    22340  53204   

  2.1.증권투자 2.2.대출금  ... D-나.2.1.증권투자 D-나.2.2.대출금 D-나.2.3.현금및예금 D-나.2.4.무역신용  \
0   3962.2  5476.2  ...            0           0             0        241.7   
1   3962.2  5476.2  ...            0           0             0        241.7   
2   3962.2  5476.2  ...            0           0             0        241.7   
3   3962.2  5476.2  ...            0           0             0        241.7   
4   3962.2  5476.2  ...            0           0             0        241.7   

  D-나.2.5.기타자산 D-나.2.6.채무상품 직접투자 D-나.2.장기 D-나.비금융기업 등  D.기타부문 

In [115]:
import pandas as pd
import requests

# 1. API로 분기 데이터 수집 (샘플 코드, 실제 호출 시 사용)
def get_bond_data():
    api_key = "T4UQ50NF71Q6Y0K4NVMZ"
    stat_code = "311Y005"
    period = "Q"
    start_date = "1994Q1"
    end_date = "2024Q4"
    url = f"https://ecos.bok.or.kr/api/StatisticSearch/{api_key}/json/kr/1/100000/{stat_code}/{period}/{start_date}/{end_date}"

    response = requests.get(url)
    data = response.json()

    if "StatisticSearch" in data and "row" in data["StatisticSearch"]:
        df = pd.DataFrame(data["StatisticSearch"]["row"])
        df = df[["ITEM_NAME1", "TIME", "DATA_VALUE"]]
        df = df.pivot(index="TIME", columns="ITEM_NAME1", values="DATA_VALUE").reset_index()
        return df
    else:
        raise ValueError("❌ 데이터가 없습니다 또는 응답 오류:", data)

# 2. 분기 데이터를 일별로 뻥튀기하는 함수
def expand_to_daily(df, value_col="대외채권"):
    df["START_DATE"] = pd.to_datetime(df["TIME"].str[:4] + '-' + df["TIME"].str[4:].replace({"Q1":"01", "Q2":"04", "Q3":"07", "Q4":"10"}) + "-01")
    daily_rows = []

    for i in range(len(df) - 1):
        start = df.loc[i, "START_DATE"]
        end = df.loc[i + 1, "START_DATE"] - pd.Timedelta(days=1)
        value = df.loc[i, value_col]
        for date in pd.date_range(start, end):
            daily_rows.append({"DATE": date, value_col: float(value)})

    # 마지막 분기 처리
    last_start = df.iloc[-1]["START_DATE"]
    last_value = df.iloc[-1][value_col]
    last_end = last_start + pd.offsets.QuarterEnd(0)
    for date in pd.date_range(last_start, last_end):
        daily_rows.append({"DATE": date, value_col: float(last_value)})

    return pd.DataFrame(daily_rows)

df_bond = get_bond_data()

if "대외채권" in df_bond.columns:
    df_daily = expand_to_daily(df_bond, value_col="대외채권")
    print(df_daily.head())
else:
    print("⚠️ '대외채권' 컬럼이 존재하지 않습니다. 아래 컬럼들을 확인해주세요:")
    print(df_bond.columns.tolist())

        DATE     대외채권
0 1994-10-01  64126.8
1 1994-10-02  64126.8
2 1994-10-03  64126.8
3 1994-10-04  64126.8
4 1994-10-05  64126.8


대외채무 일별


In [118]:
import pandas as pd

def expand_to_daily(df: pd.DataFrame, value_col: str):
    """분기 데이터를 일별로 확장하는 함수"""
    # 시작일 컬럼 생성 (Q1 → 01월, Q2 → 04월, Q3 → 07월, Q4 → 10월)
    df["START_DATE"] = pd.to_datetime(
        df["TIME"].str[:4] + "-" + df["TIME"].str[4:].replace({"Q1": "01", "Q2": "04", "Q3": "07", "Q4": "10"}) + "-01"
    )

    expanded_rows = []

    for i in range(len(df) - 1):
        start = df.loc[i, "START_DATE"]
        end = df.loc[i + 1, "START_DATE"] - pd.Timedelta(days=1)
        dates = pd.date_range(start, end)
        for date in dates:
            expanded_rows.append({
                "DATE": date,
                value_col: df.loc[i, value_col]
            })

    # 마지막 분기 처리
    last_start = df.iloc[-1]["START_DATE"]
    last_end = last_start + pd.offsets.QuarterEnd(0)
    dates = pd.date_range(last_start, last_end)
    for date in dates:
        expanded_rows.append({
            "DATE": date,
            value_col: df.iloc[-1][value_col]
        })

    df_daily = pd.DataFrame(expanded_rows)
    return df_daily

# 총합 컬럼 생성
df_debt["대외채무"] = df_debt[["1.단기", "2.장기"]].astype(float).sum(axis=1)

# 일별로 확장
df_daily_debt = expand_to_daily(df_debt, value_col="대외채무")

# 결과 확인
print(df_daily_debt.head())

        DATE     대외채무
0 1994-10-01 -16767.9
1 1994-10-02 -16767.9
2 1994-10-03 -16767.9
3 1994-10-04 -16767.9
4 1994-10-05 -16767.9


순대외채권 일별

In [119]:
def get_net_external_asset():
    api_key = "T4UQ50NF71Q6Y0K4NVMZ"
    stat_code = "311Y006"  # 순대외채권
    period = "Q"
    start_date = "1994Q1"
    end_date = "2024Q4"
    url = f"https://ecos.bok.or.kr/api/StatisticSearch/{api_key}/json/kr/1/100000/{stat_code}/{period}/{start_date}/{end_date}"

    response = requests.get(url)
    data = response.json()

    if "StatisticSearch" in data and "row" in data["StatisticSearch"]:
        df = pd.DataFrame(data["StatisticSearch"]["row"])
        df = df[["ITEM_NAME1", "TIME", "DATA_VALUE"]]
        df = df.pivot(index="TIME", columns="ITEM_NAME1", values="DATA_VALUE").reset_index()
        return df
    else:
        raise ValueError("❌ 순대외채권 데이터 없음 또는 오류:", data)

df_net_asset = get_net_external_asset()

if "순대외채권" in df_net_asset.columns:
    df_daily_net = expand_to_daily(df_net_asset, value_col="순대외채권")
    print(df_daily_net.head())
else:
    print("⚠️ '순대외채권' 컬럼명 확인:", df_net_asset.columns.tolist())

        DATE     순대외채권
0 1994-10-01  -16767.9
1 1994-10-02  -16767.9
2 1994-10-03  -16767.9
3 1994-10-04  -16767.9
4 1994-10-05  -16767.9


In [ ]:
import requests
import pandas as pd
import json

# API 파라미터 설정
# service_key = "3IY8CKY4F8FZ3ON5PFY2"
stat_code = "311Y004"
start = "1"
end = "100000"
period = "M"
start_date = "198801"
end_date = "202412"
code2 = "*AA"

url = f"https://ecos.bok.or.kr/api/StatisticSearch/{api_key}/json/kr/{start}/{end}/{stat_code}/{period}/{start_date}/{end_date}/{code2}"

response = requests.get(url)
data = response.json()

df1 = pd.DataFrame(data['StatisticSearch']['row'])
df_export_value_index = df1[['TIME', 'DATA_VALUE']] # 기준 2020년 100
df_export_value_index.rename(columns={'DATA_VALUE':'수출금액지수'},inplace=True)
df_export_value_index

In [129]:
def expand_to_daily(df_quarterly, value_col):
    df_quarterly = df_quarterly.copy()
    df_quarterly["START_DATE"] = pd.to_datetime(
        df_quarterly["TIME"].str[:4] + "-" +
        df_quarterly["TIME"].str[4:].replace({
            "Q1": "01", "Q2": "04", "Q3": "07", "Q4": "10"
        }) + "-01"
    )

    expanded = []
    for i in range(len(df_quarterly) - 1):
        start = df_quarterly.loc[i, "START_DATE"]
        end = df_quarterly.loc[i + 1, "START_DATE"] - pd.Timedelta(days=1)
        for d in pd.date_range(start, end):
            expanded.append({"DATE": d, value_col: df_quarterly.loc[i, value_col]})

    # 마지막 분기
    last_start = df_quarterly.iloc[-1]["START_DATE"]
    last_end = last_start + pd.offsets.QuarterEnd(0)
    for d in pd.date_range(last_start, last_end):
        expanded.append({"DATE": d, value_col: df_quarterly.iloc[-1][value_col]})

    return pd.DataFrame(expanded)

In [180]:
def expand_monthly_to_daily(df_monthly, date_col="DATE", value_col="외환보유액(천달러)"):
    df_monthly[date_col] = pd.to_datetime(df_monthly[date_col])
    expanded = []

    for i in range(len(df_monthly)):
        start_date = df_monthly.loc[i, date_col]
        if i < len(df_monthly) - 1:
            end_date = df_monthly.loc[i + 1, date_col] - pd.Timedelta(days=1)
        else:
            end_date = start_date + pd.offsets.MonthEnd(0)

        dates = pd.date_range(start=start_date, end=end_date)
        for d in dates:
            expanded.append({"DATE": d, value_col: df_monthly.loc[i, value_col]})

    return pd.DataFrame(expanded)

# 적용
df_foreign_reserve_daily = expand_monthly_to_daily(df_foreign_reserve_daily)

In [195]:
df_foreign_reserve_daily

,DATE,외환보유액(천달러)
0,1971-01-01,593845
1,1971-01-02,593845
2,1971-01-03,593845
3,1971-01-04,593845
4,1971-01-05,593845
...,...,...
19719,2024-12-27,415603777
19720,2024-12-28,415603777
19721,2024-12-29,415603777
19722,2024-12-30,415603777


In [163]:
df_daily_bond

,DATE,대외채권
0,1994-10-01,64126.8
1,1994-10-02,64126.8
2,1994-10-03,64126.8
3,1994-10-04,64126.8
4,1994-10-05,64126.8
...,...,...
11045,2024-12-27,1068094.7
11046,2024-12-28,1068094.7
11047,2024-12-29,1068094.7
11048,2024-12-30,1068094.7


In [164]:
df_daily_debt

,DATE,대외채무
0,1994-10-01,-16767.9
1,1994-10-02,-16767.9
2,1994-10-03,-16767.9
3,1994-10-04,-16767.9
4,1994-10-05,-16767.9
...,...,...
11045,2024-12-27,398069.4
11046,2024-12-28,398069.4
11047,2024-12-29,398069.4
11048,2024-12-30,398069.4


In [165]:
df_daily_net

,DATE,순대외채권
0,1994-10-01,-16767.9
1,1994-10-02,-16767.9
2,1994-10-03,-16767.9
3,1994-10-04,-16767.9
4,1994-10-05,-16767.9
...,...,...
11045,2024-12-27,398069.4
11046,2024-12-28,398069.4
11047,2024-12-29,398069.4
11048,2024-12-30,398069.4


In [196]:
for df in [df_daily_bond, df_daily_debt, df_daily_net, df_foreign_reserve_daily]:
    df["DATE"] = pd.to_datetime(df["DATE"])

In [198]:
df_merged = pd.merge(df_daily_bond, df_daily_debt, on="DATE", how="outer")
df_merged = pd.merge(df_merged, df_daily_net, on="DATE", how="outer")
df_merged = pd.merge(df_merged, df_foreign_reserve_daily, on="DATE", how="outer")

# 날짜 정렬
df_merged = df_merged.sort_values("DATE").reset_index(drop=True)

df_merged


,DATE,대외채권,대외채무,순대외채권,외환보유액(천달러)
0,1971-01-01,NaN,NaN,NaN,593845
1,1971-01-02,NaN,NaN,NaN,593845
2,1971-01-03,NaN,NaN,NaN,593845
3,1971-01-04,NaN,NaN,NaN,593845
4,1971-01-05,NaN,NaN,NaN,593845
...,...,...,...,...,...
19719,2024-12-27,1068094.7,398069.4,398069.4,415603777
19720,2024-12-28,1068094.7,398069.4,398069.4,415603777
19721,2024-12-29,1068094.7,398069.4,398069.4,415603777
19722,2024-12-30,1068094.7,398069.4,398069.4,415603777


In [199]:
df_merged.to_csv("all_merge.csv", index=False, encoding='utf-8-sig')

In [161]:
import pandas as pd

# 병합 (공통 기준: DATE)
df_merged = pd.merge(df_daily_bond, df_daily_debt, on="DATE", how="outer")
df_merged = pd.merge(df_merged, df_daily_net, on="DATE", how="outer")
df_merged = pd.merge(df_merged, df_foreign_reserve_daily, on="DATE", how="outer")

# 날짜 기준 정렬
df_merged = df_merged.sort_values("DATE").reset_index(drop=True)

# CSV 저장
df_merged.to_csv("daily_external_balance_merged.csv", index=False, encoding='utf-8-sig')

In [143]:
print(df_foreign_reserve.columns)

Index(['날짜', '외환보유액(천달러)'], dtype='object')
